In [27]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torch import nn
from d2l import torch as d2l
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trans = torchvision.transforms.ToTensor()

mnist_train = torchvision.datasets.FashionMNIST(root = 'E:\Deep_learning\data' , train = True , transform = trans , download = True)
mnist_test = torchvision.datasets.FashionMNIST(root = 'E:\Deep_learning\data' , train = False , transform = trans , download = True)

batch_size = 256

train_iter = DataLoader(mnist_train , batch_size , shuffle = True , num_workers = 0 , drop_last = False)
text_iter = DataLoader(mnist_test , batch_size , shuffle = False , num_workers = 0 , drop_last = False)

In [28]:
import torch.nn.functional as F

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

class Net(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fl = nn.Flatten()
        self.fc1 = nn.Linear(28*28 , 10)
    
    def forward(self, x):
        x = self.fl(x)
        x = F.softmax(self.fc1(x))
        return x

In [32]:
net = Net().to(device)
#net.apply(init_weights)
loss = nn.CrossEntropyLoss()
loss = loss.to(device)
trainer = torch.optim.SGD(net.parameters(), lr=0.1)
num_epochs = 1000
total_train_step = 0
total_test_step = 0
writer = SummaryWriter("log")

for epoch in range(num_epochs):
    #训练
    #print("----------第{}轮训练开始----------".format(epoch + 1))
    net.train()
    for data in train_iter:
        X , y = data
        X = X.to(device)
        y = y.to(device)
        y_hat = net(X)
        l = loss(y_hat , y)
        trainer.zero_grad()
        l.mean().backward()
        trainer.step()
        total_train_step += 1
        if total_train_step % 100 == 0:
            #print("训练次数{},Loss:{}".format(total_train_step,l.item()))
            writer.add_scalar("train_loss",l.item(),total_train_step)
    #测试
    net.eval()
    total_test_loss = 0
    total_accurancy = 0
    with torch.no_grad():
        for data in  text_iter:
            X , y = data
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat , y)
            total_test_loss += l.item()
            accuracy = (y_hat.argmax(1) == y).sum()
            total_accurancy += accuracy
    #print("整体测试集上的Loss：{}".format(total_test_loss))
    #print("整体测试集上的正确率：{}".format(total_accurancy / len(mnist_test)))
    writer.add_scalar("test_loss",total_test_loss,total_test_step)
    writer.add_scalar("test_accurancy",total_accurancy / len(mnist_test),total_test_step)
    total_test_step += 1
    
    torch.save(net , "E:/Deep_learning/m/net_{}.pth".format(epoch + 1))
    #print("模型已保存")

writer.close()
        

C:\Users\Jiangwang\AppData\Local\Temp\ipykernel_10728\182882822.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc1(x))
